Q: By using forward, backward, and forward-backward regressions, together with k-fold cross-validation, identify the best model for that dataset.

### About Dataset

The title of the dataset is **Estimation of Obesity Levels Based On Eating Habits and Physical Condition** and the dataset is from **https://archive.ics.uci.edu/dataset/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition**

This dataset include data for the estimation of obesity levels in individuals from the countries of Mexico, Peru and Colombia, based on their eating habits and physical condition.

The dataset contains 2111 entries with 17 features, including both numerical and categorical data. Here’s a quick rundown of the features:

1. Gender - Categorical (Male, Female)
2. Age - Numeric
3. Height - Numeric
4. Weight - Numeric
5. family_history_with_overweight - Categorical (yes, no)
6. FAVC (Frequent consumption of high caloric food) - Categorical (yes, no)
7. FCVC (Frequency of consumption of vegetables) - Numeric
8. NCP (Number of main meals) - Numeric
9. CAEC (Consumption of food between meals) - Categorical
10. SMOKE - Categorical (yes, no)
11. CH2O (Consumption of water daily) - Numeric
12. SCC (Calories consumption monitoring) - Categorical (yes, no)
13. FAF (Physical activity frequency) - Numeric
14. TUE (Time using technology devices) - Numeric
15. CALC (Consumption of alcohol) - Categorical
16. MTRANS (Transportation used) - Categorical
17. NObeyesdad - Categorical (levels of obesity)

In [23]:
import pandas as pd
import numpy as np
import itertools
import statsmodels.api as sm
import time
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error

data = pd.read_csv('data/ObesityDataSet_raw_and_data_sinthetic.csv')

print(data.head())

   Gender   Age  Height  Weight family_history_with_overweight FAVC  FCVC  \
0  Female  21.0    1.62    64.0                            yes   no   2.0   
1  Female  21.0    1.52    56.0                            yes   no   3.0   
2    Male  23.0    1.80    77.0                            yes   no   2.0   
3    Male  27.0    1.80    87.0                             no   no   3.0   
4    Male  22.0    1.78    89.8                             no   no   2.0   

   NCP       CAEC SMOKE  CH2O  SCC  FAF  TUE        CALC  \
0  3.0  Sometimes    no   2.0   no  0.0  1.0          no   
1  3.0  Sometimes   yes   3.0  yes  3.0  0.0   Sometimes   
2  3.0  Sometimes    no   2.0   no  2.0  1.0  Frequently   
3  3.0  Sometimes    no   2.0   no  2.0  0.0  Frequently   
4  1.0  Sometimes    no   2.0   no  0.0  0.0   Sometimes   

                  MTRANS           NObeyesdad  
0  Public_Transportation        Normal_Weight  
1  Public_Transportation        Normal_Weight  
2  Public_Transportation        

In [45]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [24]:
# Identify categorical columns
categorical_columns = data.select_dtypes(include=['object']).columns

In [25]:
# Apply one-hot encoding
data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

In [26]:
# Convert boolean columns to integers
bool_columns = data_encoded.select_dtypes(include=['bool']).columns
data_encoded[bool_columns] = data_encoded[bool_columns].astype(int)

In [27]:
# Standardize the features
scaler = StandardScaler()
numerical_columns = data_encoded.select_dtypes(include=['float64', 'int64']).columns
data_encoded[numerical_columns] = scaler.fit_transform(data_encoded[numerical_columns])

### Split the Dataset :

In [28]:
# response variable should be quantitative
response_variable = 'Weight'
X = data_encoded.drop(columns=[response_variable])
y = data_encoded[response_variable]

In [29]:
# Ensure all data are numeric
X = X.apply(pd.to_numeric)
y = y.apply(pd.to_numeric)

In [30]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
# Display the shape of training and testing sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1688, 28) (423, 28) (1688,) (423,)


### Define Model Selection Functions

In [31]:
# Define k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [32]:
# Define the processSubset function with cross-validation
def processSubset(feature_set, X, y, kf):
    X_const = sm.add_constant(X[list(feature_set)])
    mses = []
    r2s = []
    for train_index, val_index in kf.split(X_const):
        X_train, X_val = X_const.iloc[train_index], X_const.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        model = sm.OLS(y_train, X_train).fit()
        predictions = model.predict(X_val)
        mse = mean_squared_error(y_val, predictions)
        r2 = model.rsquared
        mses.append(mse)
        r2s.append(r2)
    avg_mse = np.mean(mses)
    avg_r2 = np.mean(r2s)
    return {"model": model, "RSS": avg_mse, "R2": avg_r2}

In [33]:
# Define getBest function
def getBest(n_predictors, X, y, kf):
    results = []
    for combo in itertools.combinations(X.columns, n_predictors):
        results.append(processSubset(combo, X, y, kf))
    models = pd.DataFrame(results)
    best_model = models.loc[models['RSS'].idxmin()]
    return best_model

In [34]:
# Backward Selection with Cross-Validation
def backward(predictors, X, y, kf):
    if len(predictors) <= 1:
        return None  # No need to do backward selection if we have 1 or 0 predictors
    
    tic = time.time()
    results = []
    for combo in itertools.combinations(predictors, len(predictors) - 1):
        results.append(processSubset(combo, X, y, kf))
    models = pd.DataFrame(results)
    best_model = models.loc[models['RSS'].idxmin()]
    toc = time.time()
    print("Processed", models.shape[0], "models on", len(predictors) - 1, "predictors in", (toc - tic), "seconds.")
    return best_model

In [35]:
# Forward Selection with Cross-Validation
def forward(predictors, X, y, kf):
    tic = time.time()
    remaining_predictors = [p for p in X.columns if p not in predictors]
    results = []
    for p in remaining_predictors:
        results.append(processSubset(predictors + [p], X, y, kf))
    models = pd.DataFrame(results)
    best_model = models.loc[models['RSS'].idxmin()]
    toc = time.time()
    print("Processed", models.shape[0], "models on", len(predictors) + 1, "predictors in", (toc - tic), "seconds.")
    return best_model

In [36]:
# Forward-Backward Selection with Cross-Validation
def forward_backward(predictors, X, y, kf):
    tic = time.time()
    results = []

    # Forward step
    remaining_predictors = [p for p in X.columns if p not in predictors]
    for p in remaining_predictors:
        results.append(processSubset(predictors + [p], X, y, kf))
    
    # Backward step, only if predictors are not empty
    if len(predictors) > 0:
        for combo in itertools.combinations(predictors, len(predictors) - 1):
            results.append(processSubset(combo, X, y, kf))
    
    models = pd.DataFrame(results)
    best_model = models.loc[models['RSS'].idxmin()]
    toc = time.time()
    print("Processed", models.shape[0], "models on", len(predictors) + 1, "predictors in", (toc - tic), "seconds.")
    return best_model

### Run Model Selection Methods:

In [37]:
# Run Backward Selection with Cross-Validation
predictors = X_train.columns.tolist()
models_bwd = pd.DataFrame(columns=["RSS", "R2", "model"], index=range(1, len(predictors)))
tic = time.time()

while len(predictors) > 1:
    best_model = backward(predictors, X_train, y_train, kf)
    if best_model is not None:
        models_bwd.loc[len(predictors) - 1] = best_model
        predictors = best_model["model"].model.exog_names[1:]

toc = time.time()
print("Total elapsed time for backward selection:", (toc - tic), "seconds.")

Processed 28 models on 27 predictors in 0.5952112674713135 seconds.
Processed 27 models on 26 predictors in 0.5412170886993408 seconds.
Processed 26 models on 25 predictors in 0.47001075744628906 seconds.
Processed 25 models on 24 predictors in 0.47474169731140137 seconds.
Processed 24 models on 23 predictors in 0.4250521659851074 seconds.
Processed 23 models on 22 predictors in 0.414229154586792 seconds.
Processed 22 models on 21 predictors in 0.4049954414367676 seconds.
Processed 21 models on 20 predictors in 0.5121030807495117 seconds.
Processed 20 models on 19 predictors in 0.38934898376464844 seconds.
Processed 19 models on 18 predictors in 0.5814323425292969 seconds.
Processed 18 models on 17 predictors in 0.36900877952575684 seconds.
Processed 17 models on 16 predictors in 0.4110419750213623 seconds.
Processed 16 models on 15 predictors in 0.3821375370025635 seconds.
Processed 15 models on 14 predictors in 0.1862201690673828 seconds.
Processed 14 models on 13 predictors in 0.150

In [38]:
# Run Forward Selection with Cross-Validation
predictors = []
models_fwd = pd.DataFrame(columns=["RSS", "R2", "model"], index=range(1, len(X_train.columns) + 1))
tic = time.time()

for i in range(1, len(X_train.columns) + 1):
    best_model = forward(predictors, X_train, y_train, kf)
    models_fwd.loc[i] = best_model
    predictors = best_model["model"].model.exog_names[1:]

toc = time.time()
print("Total elapsed time for forward selection:", (toc - tic), "seconds.")

Processed 28 models on 1 predictors in 0.19500041007995605 seconds.
Processed 27 models on 2 predictors in 0.18939661979675293 seconds.
Processed 26 models on 3 predictors in 0.21008825302124023 seconds.
Processed 25 models on 4 predictors in 0.22831416130065918 seconds.
Processed 24 models on 5 predictors in 0.21955084800720215 seconds.
Processed 23 models on 6 predictors in 0.24396014213562012 seconds.
Processed 22 models on 7 predictors in 0.23849153518676758 seconds.
Processed 21 models on 8 predictors in 0.1661384105682373 seconds.
Processed 20 models on 9 predictors in 0.17100262641906738 seconds.
Processed 19 models on 10 predictors in 0.17186570167541504 seconds.
Processed 18 models on 11 predictors in 0.1820216178894043 seconds.
Processed 17 models on 12 predictors in 0.15642547607421875 seconds.
Processed 16 models on 13 predictors in 0.16955161094665527 seconds.
Processed 15 models on 14 predictors in 0.15625262260437012 seconds.
Processed 14 models on 15 predictors in 0.320

In [39]:
# Run Forward-Backward Selection with Cross-Validation
predictors = []
models_fwd_bwd = pd.DataFrame(columns=["RSS", "R2", "model"], index=range(1, len(X_train.columns) + 1))
tic = time.time()

for i in range(1, len(X_train.columns) + 1):
    best_model = forward_backward(predictors, X_train, y_train, kf)
    models_fwd_bwd.loc[i] = best_model
    predictors = best_model["model"].model.exog_names[1:]

toc = time.time()
print("Total elapsed time for forward-backward selection:", (toc - tic), "seconds.")

Processed 28 models on 1 predictors in 0.16898584365844727 seconds.
Processed 28 models on 2 predictors in 0.18557357788085938 seconds.
Processed 28 models on 3 predictors in 0.21342253684997559 seconds.
Processed 28 models on 4 predictors in 0.2431659698486328 seconds.
Processed 28 models on 5 predictors in 0.26268458366394043 seconds.
Processed 28 models on 6 predictors in 0.2868683338165283 seconds.
Processed 28 models on 7 predictors in 0.30193138122558594 seconds.
Processed 28 models on 8 predictors in 0.24024343490600586 seconds.
Processed 28 models on 9 predictors in 0.24741029739379883 seconds.
Processed 28 models on 10 predictors in 0.2577788829803467 seconds.
Processed 28 models on 11 predictors in 0.2643885612487793 seconds.
Processed 28 models on 12 predictors in 0.2595202922821045 seconds.
Processed 28 models on 13 predictors in 0.2827112674713135 seconds.
Processed 28 models on 14 predictors in 0.2857072353363037 seconds.
Processed 28 models on 15 predictors in 0.51666879

### Evaluate and Determine the Best Model

In [40]:
# Evaluate the models
def evaluate_models(models):
    print("Number of Predictors | RSS        | R2")
    for i in range(1, len(models) + 1):
        print(f"{i:19} | {models.loc[i, 'RSS']:.5f} | {models.loc[i, 'R2']:.5f}")

print("-----------------")
print("Backward Selection:")
print("-----------------")
evaluate_models(models_bwd)

print("-----------------")
print("Forward Selection:")
print("-----------------")
evaluate_models(models_fwd)

print("-----------------")
print("Forward-Backward Selection:")
print("-----------------")
evaluate_models(models_fwd_bwd)

-----------------
Backward Selection:
-----------------
Number of Predictors | RSS        | R2
                  1 | 0.68348 | 0.31237
                  2 | 0.37924 | 0.61937
                  3 | 0.25771 | 0.74196
                  4 | 0.14774 | 0.85214
                  5 | 0.09949 | 0.90082
                  6 | 0.05780 | 0.94209
                  7 | 0.03979 | 0.96017
                  8 | 0.03912 | 0.96084
                  9 | 0.03863 | 0.96140
                 10 | 0.03820 | 0.96183
                 11 | 0.03788 | 0.96215
                 12 | 0.03776 | 0.96229
                 13 | 0.03765 | 0.96244
                 14 | 0.03758 | 0.96252
                 15 | 0.03754 | 0.96262
                 16 | 0.03749 | 0.96271
                 17 | 0.03744 | 0.96282
                 18 | 0.03744 | 0.96283
                 19 | 0.03746 | 0.96284
                 20 | 0.03747 | 0.96292
                 21 | 0.03749 | 0.96293
                 22 | 0.03751 | 0.96293
                 23 | 0.0

#### Insights:
The stability of RSS and R² values from 17 predictors onwards in Forward-Backward Selection indicates that the model has likely reached its optimal complexity. Additional predictors are not contributing to better model performance.

In [41]:
#Determine the best model
def determine_best_model(models):
    best_index = models['RSS'].idxmin()
    best_model = models.loc[best_index, "model"]
    print(f"Best Model with {best_index} predictors")
    print(f"RSS: {models.loc[best_index, 'RSS']:.5f}, R2: {models.loc[best_index, 'R2']:.5f}")
    return best_model

In [42]:
print("Determining the Best Model from Forward-Backward Selection:")
best_model = determine_best_model(models_fwd_bwd)

Determining the Best Model from Forward-Backward Selection:
Best Model with 17 predictors
RSS: 0.03744, R2: 0.96282


#### Insights:

By focusing on stepwise selection methods, which include forward selection, backward selection, and a combination of both (forward-backward selection), is a practical approach because it balances computational efficiency with model performance.

Best Model Characteristics:

- Number of Predictors: The best model uses 17 predictors.
- Residual Sum of Squares (RSS): 0.03744. RSS is a measure of the discrepancy between the observed and predicted values. A lower RSS indicates a better fit of the model to the data.
- R-squared (R²): 0.96282. R² indicates the proportion of the variance in the dependent variable that is predictable from the independent variables. An R² value of 0.96282 means that approximately 96.28% of the variance in the response variable (Weight) is explained by the predictors in the model.

In [43]:
# Print the parameters of the best model
print(best_model.params)

const                            -1.379973
NObeyesdad_Obesity_Type_III       2.790548
NObeyesdad_Obesity_Type_II        2.272819
Height                            0.346073
NObeyesdad_Obesity_Type_I         1.680967
NObeyesdad_Overweight_Level_II    1.239459
NObeyesdad_Overweight_Level_I     0.978054
NObeyesdad_Normal_Weight          0.554750
CAEC_Frequently                   0.098027
FAF                               0.025364
NCP                              -0.024388
CALC_Sometimes                    0.047349
Age                              -0.024398
MTRANS_Public_Transportation     -0.054816
CAEC_no                           0.065153
MTRANS_Walking                   -0.089349
FAVC_yes                         -0.032663
TUE                               0.007518
dtype: float64


#### Insights:
Significant Predictors:
- NObeyesdad_Obesity_Type_III: 2.790548
- NObeyesdad_Obesity_Type_II: 2.272819
- NObeyesdad_Obesity_Type_I: 1.680967
- NObeyesdad_Overweight_Level_II: 1.239459
- NObeyesdad_Overweight_Level_I: 0.978054
- NObeyesdad_Normal_Weight: 0.554750

These coefficients indicate the impact of different obesity and weight categories on the response variable.

- Obesity and Weight Categories: The largest coefficients are associated with the obesity and weight categories, indicating these factors have the most significant impact on the response variable.
- Height and Physical Activity: Height and physical activity (FAF) are positively correlated with the response variable.
- Age and Meal Patterns: Age and meal patterns (NCP) show a slight negative association, indicating that older age and more frequent meals might slightly decrease the response variable.
- Transportation and Alcohol Consumption: Modes of transportation and alcohol consumption patterns have smaller but notable effects.

### Evaluating the Model on Test Data

In [46]:
# Add a constant to the test set
X_test_const = sm.add_constant(X_test)

# Make predictions on the test set using the best model
y_pred = best_model.predict(X_test_const[list(best_model.params.index)])

# Calculate performance metrics on the test set
test_mse = mean_squared_error(y_test, y_pred)
test_r2 = best_model.rsquared

print(f"Test MSE: {test_mse:.5f}")
print(f"Test R²: {test_r2:.5f}")

Test MSE: 0.03927
Test R²: 0.96291


#### Insights:
1. The low Test MSE indicates that the model makes accurate predictions on the test data.
2. The high Test R² shows that the model explains most of the variability in the response variable.

The model has demonstrated excellent predictive performance on the test data, as evidenced by the low Test MSE and high Test R². These results suggest that the model is both accurate and generalizes well to unseen data, making it a reliable tool for prediction and analysis in the given context.